In [249]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [250]:
# Loading all data bases 

branchements = pd.read_csv('../data_SUEZ/branchements.csv')
canalisations = pd.read_csv('../data_SUEZ/canalisations.csv')
evenements_train = pd.read_csv('../data_SUEZ/evenements_train.csv')
meteo_AlpesMaritimes = pd.read_csv('../data_SUEZ/meteo_AlpesMaritimes_pre_traite.csv')
nb_ev_train = pd.read_csv('../data_SUEZ/nb_ev_train.csv')
permut_train = pd.read_csv('../data_SUEZ/permut_train.csv')
stations_AlpesMaritimes = pd.read_csv('../data_SUEZ/stations_AlpesMaritimes.csv')

# Preprocessing

## Jointure

In [251]:
# Renaming columns

evenements_train.columns= ['GID_fuite', 'CODINSEE_fuite', 'date_fuite', 'GID_pose', 'COMPO_P', 'geometry_fuite']
branchements.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']
canalisations.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']

In [252]:
evenements_train['fuite'] = 1

In [253]:
# Différencier les GID des branchements et des canalisations

evenements_train['ajout_branch'] = '_b'
evenements_train['ajout_can'] = '_c'
evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'ajout_branch']
evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'ajout_can']
evenements_train.drop('ajout_branch', axis=1, inplace=True)
evenements_train.drop('ajout_can', axis=1, inplace=True)

branchements['ajout_branch'] = '_b'
branchements.GID_pose = branchements.GID_pose.map(str) + branchements.ajout_branch
branchements.drop('ajout_branch', axis=1, inplace=True)

canalisations['ajout_can'] = '_c'
canalisations.GID_pose = canalisations.GID_pose.map(str) + canalisations.ajout_can
canalisations.drop('ajout_can', axis=1, inplace=True)

In [254]:
# Merge elements on evenements (creation BDD complète)

elements = pd.concat([canalisations, branchements])
data = pd.merge(elements, evenements_train, on = 'GID_pose', how = 'left')

## Feature engineering

In [255]:
# Utiliser le code INSEE de la fuite quand on l'a plutot que celui de la pose
data.ix[~data.CODINSEE_fuite.isnull(), 'CODINSEE_pose'] = data.ix[~data.CODINSEE_fuite.isnull(), 'CODINSEE_fuite']
data.drop('CODINSEE_fuite', axis = 1, inplace = True)

In [256]:
# Convertir âge

def to_day(x):
    if (str(x) == 'NaT'):
        return np.nan
    else:
        return x.days

data['age'] = pd.to_datetime('2016-12-31') - pd.to_datetime(data['date_pose'])
data.age = data.age.map(to_day)

In [257]:
data.ix[data.fuite.isnull(), 'fuite'] = 0

In [258]:
# Encode categorical variables

le = LabelEncoder()
mapping = dict()
for col, dtype in zip(data.columns, data.dtypes):
    if dtype == 'object':
        data[col] = data[col].apply(lambda s: str(s))
        # Replace 0 and NaNs with unique label : 'None'
        data[col] = data[col].where(~data[col].isin(['0', 'nan']), 'None')
        data[col] = le.fit_transform(data[col])
        mapping[col] = dict(zip(le.inverse_transform(data[col].unique()), data[col].unique()))

In [259]:
# Replace NaNs with 0
data = data.apply(lambda x: x.fillna(x.mean()),axis=0)

# Training

## Séparation submit et jeu d'entrainement

In [260]:
# Séparer le dataset d'entrainment du dataset de test
test_insee = [6003, 6004, 6016, 6030, 6070, 6095, 6108, 6112, 6116, 6138, 6152]
data_known = data[~data['CODINSEE_pose'].isin(test_insee)]
submit = data[data['CODINSEE_pose'].isin(test_insee)]

In [261]:
submit = submit.reset_index().ix[:,1:18]
submit_CODINSEE = submit.CODINSEE_pose

In [262]:
# Remove unnecessary columns
fuite_columns = ['GID_fuite', 'date_fuite', 'geometry_fuite']
id_columns = ['GID_pose', 'CODINSEE_pose']
a_predir = ['fuite']
autres = ['GEORIENT', 'date_pose', 'geometry_pose', 'ETAGE', 'DIAMINT', 'DIAMEXT']
mask = id_columns + fuite_columns + a_predir + autres  # geometry_pose pas encore exploitable

X = data_known.drop(mask, axis = 1)
y = data_known['fuite']

X_submit = submit.drop(mask, axis = 1)

## Random Forest

In [263]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [264]:
y_submit = clf.predict(X_submit)

# Submitting

In [265]:
def export_submission(mypred):
    mypred = pd.DataFrame(mypred)
    
    submit['pred'] = mypred
    submit['CODEINSEE'] = submit_CODINSEE
    
    mysubmission = pd.DataFrame({'count' : submit.groupby( [ 'CODEINSEE'] )['pred'].sum()}).reset_index()
    mysubmission.columns = ['Codinsee', 'nb_ev'] 
    mysubmission.to_csv('nb_ev_train.csv', sep=',', index=False)

    mysub_rank = mysubmission.sort_values(by = 'nb_ev', ascending=False).ix[:,0]
    mysub_rank.to_csv('permut_train.csv', sep=',', index=False)

In [245]:
export_submission(y_submit)

# Metric

In [267]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

# Aggrégé par communes

In [268]:
communes_train = [6007, 6008, 6010, 6028, 6029, 6038, 6068, 6071, 6079, 6084, 6085, 6089, 6090, 6094, 6069, 6134, 6154, 6155, 6161, 83118]
communes_test = [6003, 6004, 6016, 6030, 6070, 6095, 6108, 6112, 6116, 6138, 6152]

In [276]:
communes = pd.DataFrame(data.CODINSEE_pose.unique())
communes.columns = ['CODINSEE']

In [289]:
communes['long_br'] = branchements.groupby( [ 'CODINSEE_pose'] ).sum().ix[:,4]
communes['long_ca'] = canalisations.groupby( [ 'CODINSEE_pose'] ).sum().ix[:,4]

In [290]:
communes

,CODINSEE,long_br,long_ca
0,6134.0,NaN,NaN
1,6154.0,NaN,NaN
2,6116.0,NaN,NaN
3,6028.0,NaN,NaN
4,6003.0,NaN,NaN
5,6070.0,NaN,NaN
6,6095.0,NaN,NaN
7,6071.0,NaN,NaN
8,6069.0,NaN,NaN
9,6007.0,NaN,NaN
